In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
start="2022-01-01"
end="2026-01-01"
market_ticker = "SPY"
tickers = [
    "GOOGL", "NVDA",
    "PLTR", "WFC",
    "DIS", "LUV", "PFE",
    "COKE", "CAT"
]
tickers.append(market_ticker)


In [ ]:
def download_data(stocks, start, end):
  df = yf.download(stocks, start=start, end=end, auto_adjust=False)[["Close"]]
  market_col = ("Close", market_ticker)
  market = df[market_col]
  df = df.drop(market_col, axis=1)
  return (df, market)

In [ ]:
df, market = download_data(tickers, start, end)

In [ ]:
def get_beta(returns, market_returns):
    betas = {}

    for col in returns.columns:
        aligned = returns[col].align(market_returns, join="inner")
        s, m = aligned

        cov = np.cov(s, m)
        betas[col] = cov[0, 1] / cov[1, 1]

    return pd.Series(betas, name="Beta")

def matrix_beta(returns, market_returns):
    returns, market_returns = returns.align(market_returns, join="inner", axis=0)

    Y = returns.to_numpy()                  
    x = market_returns.to_numpy()[:, None]  

    X = np.column_stack([np.ones(len(x)), x])
    beta = np.linalg.lstsq(X, Y, rcond=None)[0]
    betas = pd.Series(beta[1], index=returns.columns, name="Beta")
    return betas


In [ ]:
log_ret = np.log(df["Close"]/df["Close"].shift()).dropna()
#betas = get_beta(log_ret, market)
betas = matrix_beta(log_ret, market)

In [ ]:
units = np.array([10]*(len(tickers)-1))
prices = df["Close"].iloc[-1].to_numpy()
value = units*prices
w = [val/sum(value) for val in value]


In [ ]:
Portfolio = pd.DataFrame(
    {
        "tickers": tickers[:-1],
        "Type": "S",
        "direction": "long",
        "stock prices": prices,
        "prices": prices,
        "units": units,
        "value": value,
        "beta": betas,
        "weight": w,
        "weighted beta": betas*w/sum(betas*w)
    }
)
